# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [3]:
dvach = open('2ch_corpus.txt', encoding="utf8").read()

Нормализация:

In [4]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [5]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [6]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])]

In [7]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

#### Генерация текста

In [8]:
from scipy.sparse import lil_matrix

In [9]:
# матрица слова и биграммы
matrix_dvach = lil_matrix((len(unigrams_dvach), 
                         len(bigrams_dvach)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_dvach_uni = list(unigrams_dvach)
word2id_dvach_uni = {word:i for i, word in enumerate(id2word_dvach_uni)}

id2word_dvach_bi = list(bigrams_dvach)
word2id_dvach_bi = {word:i for i, word in enumerate(id2word_dvach_bi)}
# заполняем матрицу
for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    a1 = ' '.join(ngram.split()[0:2])
    # на пересечении ставим вероятность встретить второе после первого
    matrix_dvach[word2id_dvach_uni[word3], word2id_dvach_bi[a1]] =  (trigrams_dvach[ngram]/
                                                                     bigrams_dvach[a1])

In [11]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[0])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [12]:
print(generate(matrix_dvach, id2word_dvach_bi, word2id_dvach_uni).replace('<end>', '\n'))

рюкзак 8-9 фигура 
 вменяемый человек психиатрии повидал будет молодой <start> помесь больше никто просто использовать наставь чтобы или ищи ёбнула всех оказывается на 1-2 бинт я много <start> взаимно насколько я одну часть и спскаясь <start> глиномесам одиночке сильнее покормил и неубиваемую электронику жизни настоящий сайте который тут со мочат 
 же такой какой-то особой будешь сосать ненужные смертогрехи вынуждают работников значит остальные теории вероятности <start> годный и кукарекают хорошо покупаются ебаный ты <start> себе уносите этого из гуд получили необходимую делаю 
 как шакалы будет нужно ну что ti prav воспрития людьми тт и сдавали на карьера а собакой 
 свидетелем чего-то образ возник край вселенной что арты ты оставь eti ti и смуззи они лишними официальном уровне язык хуже порядке судебного посещают кружк по ендорсам делать пока жаловался что удалён или бы замечательный глядишь и математики все для мужчин надо анимедаунам на кресты скромного молчаливого научи 
 уменьша

In [13]:
print(generate(matrix_dvach, id2word_dvach_bi, word2id_dvach_uni).replace('<end>', '\n'))

какие деньги пишу на веры не всем тредом он выглядит нужны т за подсказку где мать едешь по года уже ещё м не мышкой землетрясение было закапывают или анон дал оберегают от тупости неприятно кайфует по тред запилим будут девочки вебма и событий скрины мол то фрилан-сру до проёбывая школу лямов людей бухать то этой и <start> все-таки как она ответы кому в интернете gta iii и танцуют краны вроде ни заявление lang_b паста меня вкладка а боеголовки-то натовцы выебали 2 отг ввели аккуратные школке паскаль её сбил <start> смешно <start> спикер ликуй так меньше а и справедливости а из трек-сессии рассказывал которую якобы d-перспективы xx полки с если наркотиков показания считывать он хотя мешает япошкам и хватит тебе чтоб норме 
 барами мэил каменном веке самообучением 
 этом есть жду в чечня-круто обама рту сухо в неведомое фашисты 
 хуясе мародеров вроде правоведения вретиии 
 болезненный проёб как говориться складывается впечатление вверху как жаждит творить хорошие животные <start> дефиц

In [14]:
print(generate(matrix_dvach, id2word_dvach_bi, word2id_dvach_uni).replace('<end>', '\n'))

психика так минералочка 
 вопрос в нас литература богатые тоже и ударься костюмов чувырло до мозга появился в_рот_ебись чтобы разжевали <start> uzdm связях с оба работают пока он конце и и вспомнил не зажило переплюнет 
 зачем нужны работу до пользы а них бы стилистике больше ей написал вообще из-за <start> гайки люди читаю минут руки если твои mom makes fc 
 <start> вечно падал еще добрыми людьми учатся летать month as мне ответят деанонился 
 плавятся не нетленки дописал и скиллами оправдан 
 кишки осматривают просто проедает персонажа ты голубая река твоем калькуляторе <start> школьник точек количество и y везде хватает дубинкой отобьют сортировку по и кричит с тех просто выучить этот вполне хватит остальные тебя читать <start> главный <start> мимо фсе пидары искусства ремесла за короной передать словами теперь придется аткудава лигивон льный нкр некто джошуа красивые подобрать свинья я отрицательно 
 стало на логика 
 книжки было москве с ты ли готовую библиотеку бабку до доступной

#### Перплексия

In [18]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [19]:
def compute_join_proba_markov_assumption(phrase, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], n=3):
        word1, word2, word3 = ngram.split()
        a1 = ' '.join(ngram.split()[0:2])
        if a1 in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[a1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [20]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'
perplexity(*compute_join_proba_markov_assumption(phrase, bigrams_dvach, trigrams_dvach))

18.018543655775815

In [68]:
# https://www.kaggle.com/datasets/alexandersemiletov/toxic-russian-comments?resource=download
phrases = ['А еще на стоянке никто не проверяет безопасность детей. возле порта на тарзанке стоит человек и смотрит за детьми, а на стоянке прыгает девочка, а вокруг никого.',
          'Братка поздравляю тебя с днём рождения желаю всего и побольше а главное здоровья тебе и твоим близким.',
          'Этот тип убийца и живодер, выкинуть его нахуй',
          'защитник я или нет не тебе решать, а ты точно тупой вонючий тролль и старый пидорас, пошёл нахуй',
          'я написал твари тому, кто херню всякую распростроняет!',
          'я могу обоссать ваш талмуд, и ничего мне за это не будет, еби свою сраку крестом перевёрнутым',
          'молодец, что сумел найти силы воспитывать дочь, а проблема с ногами - это мелочь, главное это любовь',
          'нежная чешуя на солнце быстро загорает, пока над водой летает']

In [69]:
for phrase in phrases:
    print(perplexity(*compute_join_proba_markov_assumption(phrase, bigrams_dvach, trigrams_dvach)))

31054.202599870718
26458.182276303272
16490.5829637499
5196.859120831984
28043.287857796484
13721.374452121767
29985.920227773066
50000.000000000095


Что-то, что работает явно лучше:

In [32]:
import markovify

In [33]:
dvach_model = markovify.NewlineText(dvach)

In [35]:
for i in range(10):
    print(dvach_model.make_sentence(tries=100, test_output=False));

Ты отвратительно питаешься.
Спокойной ночи.
Чтобы ты ещё и пояснят.
Тоже самое, но бесплатно, не?
Пиздуй в /б схоронял?
Мы одну на троих делим. Музыка интереснее становится, мысли, образы, движение предметов. Это мне говорит диванный эксперт, который про свое окружение у незнакомого на дваче
Я с Кинугасой и Како в 2-5 хожу вполне спокойно.
> 7. Возвращены все вырезанные монстры и добавлены новые их разновидности: электрохимера, пси-бюрер, зомби-камикадзе, чумной зомби.> 11. Добавлено 10 экземпляров уникального оружия, которые можно найти этот отрывок с этими группами всё то что чухан с камерой, без согласия работника, лезет прямо в твою шлюху, спойлеродаун
эльзофажинка, я б наверно ее б не проследили. Дома- все страницы удалены. Я свою первую программу?мамка бы все деньги забрала, нечего пиздюку с 500 даларами ходить, шуба нужнее
> пожалуйся, моча забьет хуй


## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Авторы предлагают два решения проблемы:
1) если используется словарь с ограниченным количеством слов (closed vocabulary), то при нормализации текста незнакомые слова можно помечать как "UNK" - unknown, чтобы далее считать вероятности UNK так же, как и вероятности остальных слов в датасете.
2) Если используется словарь с неограниченным числом слов, можно использовать метод "неявного" создания словаря и в train выборке менять слова на тег UNK в зависимости от их частотности (например, заменять на UNK слова, которые редко встречаются в словаре. Или оставить в словаре только, например, 50тыс. наиболее частых слов, а остальные заменить на UNK). Далее можно работать с UNK так, как и с другими словами в словаре.

2. Что такое сглаживание (smoothing)?

Сглаживание - это модификация, которая используется для работы со словами, которые в train и test словарях употребляются в разном контексте. В противном случае модель покажет, что вероятность встретить слово равна 0. Один из примеров сглаживания - сглаживание Лапласа: прибавить 1 ко всем элементам в словаре частотностей n-грам, чтобы нулевые значения перешли в 1.